In [10]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import time
 


In [ ]:

# Ruta a la carpeta con subcarpetas para cada emoción
dataset_dir = "C:\\Users\\angel\\OneDrive\\Documentos\\Proj_Alg\\Proj_Alg\\Proj_Alg\\dataset"
image_size = (100, 100)  # Tamaño de las imágenes para PCA

# Cargar el clasificador de detección facial
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Listas para almacenar las imágenes y etiquetas
images = []
labels = []

# Cargar imágenes y etiquetas
for emotion_name in os.listdir(dataset_dir):
    print(f"Cargando imágenes de la emoción: {emotion_name}...")
    emotion_dir = os.path.join(dataset_dir, emotion_name)
    if os.path.isdir(emotion_dir):
        for img_name in os.listdir(emotion_dir):
            img_path = os.path.join(emotion_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)
                for (x, y, w, h) in faces:
                    face = img[y:y+h, x:x+w]  # Recorta la región facial
                    img_resized = cv2.resize(face, image_size)  # Redimensionar la cara
                    images.append(img_resized.flatten())  # Aplanar la imagen
                    labels.append(emotion_name)  # Usa el nombre de la carpeta como etiqueta

# Convertir listas a matrices de numpy
images = np.array(images)
labels = np.array(labels)

# Aplicar PCA para reducción de dimensionalidad (eigenfaces)
n_components = 20  # Número de componentes principales (ajustable)
pca = PCA(n_components=n_components, whiten=True)
X_pca = pca.fit_transform(images)

# Dividir en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, labels, test_size=0.25, random_state=42)

# Entrenar el clasificador KNN
knn_clf = KNeighborsClassifier(n_neighbors=3, algorithm='ball_tree', weights='distance')
knn_clf.fit(X_train, y_train)

# Guardar el modelo entrenado y el objeto PCA
with open("C:\\Users\\angel\\OneDrive\\Documentos\\Proj_Alg\\modelo_emociones_knn.pkl", "wb") as model_file:
    pickle.dump((knn_clf, pca), model_file)

# Evaluar el modelo
y_pred = knn_clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred, target_names=np.unique(labels)))


Cargando imágenes de Emiliano...
Cargando imágenes de Paola...
Cargando imágenes de Pedro...
Cargando imágenes de Ramses...
Accuracy: 0.90


In [16]:

# Cargar el clasificador KNN y el modelo PCA
with open("C:\\Users\\angel\\OneDrive\\Documentos\\Proj_Alg\\modelo_eigenfaces_knn.pkl", "rb") as model_file:
    knn_clf, pca = pickle.load(model_file)

# Configuración de la captura de video y el clasificador de rostros
cap = cv2.VideoCapture(0)  # 0 es para la cámara integrada
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
#Variables para calcular FPS
prev_frame_time = 0
new_frame_time = 0
# Tamaño de la imagen para PCA
image_size = (100, 100)

while True:
    # Capturar un frame de video
    ret, frame = cap.read()
    if not ret:
        break
    # Calcular FPS
    new_frame_time = time.time()
    fps = int(1 / (new_frame_time - prev_frame_time))
    prev_frame_time = new_frame_time
    # Añadir texto a la ventana
    cv2.putText(frame, "Presiona 'q' para salir", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"FPS: {fps}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    # Convertir a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros en el frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extraer y redimensionar la región de interés (ROI)
        face_roi = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face_roi, image_size)
        face_flattened = face_resized.flatten().reshape(1, -1)

        # Aplicar PCA a la imagen de la cara
        face_pca = pca.transform(face_flattened)

        # Realizar la predicción con KNN
        prediction = knn_clf.predict(face_pca)
        label = prediction[0]

        # Dibujar el cuadro delimitador y la etiqueta
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # Mostrar el frame con las detecciones y predicciones
    cv2.imshow("Reconocimiento Facial en Tiempo Real", frame)

    # Salir al presionar 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar ventanas
cap.release()
cv2.destroyAllWindows()
